# Visualizar dados da malha

In [1]:
import xmltodict
import json
import pandas as pd


In [2]:
with open('malha.net.xml', 'rb') as f:
    malha_xml = xmltodict.parse(
        f, attr_prefix='', cdata_key='', dict_constructor=dict)


In [3]:
print(json.dumps(malha_xml['net']['edge'], indent=4, ensure_ascii=False))


[
    {
        "id": "gneE0",
        "from": "gneJ31",
        "to": "gneJ32",
        "name": "Av.Ver.Abrahão João Francisco",
        "priority": "4",
        "spreadType": "center",
        "lane": [
            {
                "id": "gneE0_0",
                "index": "0",
                "speed": "16.00",
                "length": "210.12",
                "shape": "730392.09,7019639.90 730560.53,7019765.32"
            },
            {
                "id": "gneE0_1",
                "index": "1",
                "speed": "16.00",
                "length": "210.12",
                "shape": "730390.17,7019642.46 730558.62,7019767.89"
            }
        ],
        "param": [
            {
                "key": "vmax",
                "value": "16"
            },
            {
                "key": "vmin",
                "value": "11.2"
            }
        ]
    },
    {
        "id": "gneE1",
        "from": "gneJ32",
        "to": "gneJ33",
        "name": "Av.Ver.Abr